# Version pandas with deltalake

This notebook works with the `pandas-deltalake` environment.

In [24]:
import os

import pandas as pd
from deltalake import DeltaTable
from deltalake.writer import write_deltalake

## Create Delta Lake

In [25]:
df = pd.DataFrame({"x": [1, 2, 3]})

In [26]:
df

x
0  1
1  2
2  3

In [27]:
os.makedirs("tmp/some_delta_lake", exist_ok=True)

In [28]:
write_deltalake("tmp/some_delta_lake", df)

In [29]:
dt = DeltaTable("tmp/some_delta_lake")

In [30]:
dt.to_pandas()

x
0  1
1  2
2  3

In [31]:
!tree tmp/some_delta_lake

tmp/some_delta_lake
├── 0-c095d563-6d54-47bb-a8ee-c6d6148e8d38-0.parquet
└── _delta_log
    └── 00000000000000000000.json

1 directory, 2 files


In [32]:
!jq . tmp/some_delta_lake/_delta_log/00000000000000000000.json

{
  "commitInfo": {
    "delta-rs": "0.8.0",
    "timestamp": 1683225239524
  }
}
{
  "protocol": {
    "minReaderVersion": 1,
    "minWriterVersion": 1
  }
}
{
  "metaData": {
    "id": "006bc626-3d8a-4ee1-b141-07e772e360ce",
    "name": null,
    "description": null,
    "format": {
      "provider": "parquet",
      "options": {}
    },
    "schemaString": "{\"type\":\"struct\",\"fields\":[{\"name\":\"x\",\"type\":\"long\",\"nullable\":true,\"metadata\":{}}]}",
    "partitionColumns": [],
    "createdTime": 1683225239523,
    "configuration": {}
  }
}
{
  "add": {
    "path": "0-c095d563-6d54-47bb-a8ee-c6d6148e8d38-0.parquet",
    "size": 1654,
    "partitionValues": {},
    "modificationTime": 1683225239523,
    "dataChange": true,
    "stats": "{\"numRecords\": 3, \"minValues\": {\"x\": 1}, \"maxValues\": {\"x\": 3}, \"nullCount\": {\"x\": 0}}",
    "tags": null
  }
}


## Append to Delta Lake

In [33]:
df2 = pd.DataFrame({"x": [8, 9, 10]})

In [34]:
write_deltalake("tmp/some_delta_lake", df2, mode="append")

In [35]:
!tree tmp/some_delta_lake

tmp/some_delta_lake
├── 0-c095d563-6d54-47bb-a8ee-c6d6148e8d38-0.parquet
├── 1-75c05f5e-116c-4d87-aebf-1b80ad497d66-0.parquet
└── _delta_log
    ├── 00000000000000000000.json
    └── 00000000000000000001.json

1 directory, 4 files


In [36]:
DeltaTable("tmp/some_delta_lake").to_pandas()

x
0   1
1   2
2   3
3   8
4   9
5  10

## Overwrite Delta Lake

In [37]:
df3 = pd.DataFrame({"x": [55, 66, 77]})

In [38]:
df3

x
0  55
1  66
2  77

In [39]:
write_deltalake("tmp/some_delta_lake", df3, mode="overwrite")

In [40]:
!tree tmp/some_delta_lake

tmp/some_delta_lake
├── 0-c095d563-6d54-47bb-a8ee-c6d6148e8d38-0.parquet
├── 1-75c05f5e-116c-4d87-aebf-1b80ad497d66-0.parquet
├── 2-b5214974-a0a4-42cf-84ba-f8d4efcb9131-0.parquet
└── _delta_log
    ├── 00000000000000000000.json
    ├── 00000000000000000001.json
    └── 00000000000000000002.json

1 directory, 6 files


In [41]:
DeltaTable("tmp/some_delta_lake").to_pandas()

x
0  55
1  66
2  77

In [42]:
!jq . tmp/some_delta_lake/_delta_log/00000000000000000002.json

{
  "add": {
    "path": "2-b5214974-a0a4-42cf-84ba-f8d4efcb9131-0.parquet",
    "size": 1654,
    "partitionValues": {},
    "modificationTime": 1683225508280,
    "dataChange": true,
    "stats": "{\"numRecords\": 3, \"minValues\": {\"x\": 55}, \"maxValues\": {\"x\": 77}, \"nullCount\": {\"x\": 0}}",
    "tags": null
  }
}
{
  "remove": {
    "path": "0-c095d563-6d54-47bb-a8ee-c6d6148e8d38-0.parquet",
    "deletionTimestamp": 1683225508281,
    "dataChange": true,
    "extendedFileMetadata": false,
    "partitionValues": {},
    "size": 1654,
    "tags": null
  }
}
{
  "remove": {
    "path": "1-75c05f5e-116c-4d87-aebf-1b80ad497d66-0.parquet",
    "deletionTimestamp": 1683225508281,
    "dataChange": true,
    "extendedFileMetadata": false,
    "partitionValues": {},
    "size": 1654,
    "tags": null
  }
}
{
  "commitInfo": {
    "timestamp": 1683225508281,
    "operation": "WRITE",
    "operationParameters": {
      "partitionBy": "[]",
      "mode": "Overwrite"
    },
    "clientV

## Confirm other versions are still accessible

In [43]:
DeltaTable("tmp/some_delta_lake", version=0).to_pandas()

x
0  1
1  2
2  3

In [44]:
DeltaTable("tmp/some_delta_lake", version=1).to_pandas()

x
0   1
1   2
2   3
3   8
4   9
5  10

In [45]:
DeltaTable("tmp/some_delta_lake").to_pandas()

x
0  55
1  66
2  77

In [46]:
DeltaTable("tmp/some_delta_lake", version=2).to_pandas()

x
0  55
1  66
2  77

## Schema enforcement prevents bad appends

In [47]:
df4 = pd.DataFrame({"y": [111, 222]})

In [48]:
write_deltalake("tmp/some_delta_lake", df4, mode="append")

ValueError: Schema of data does not match table schema
Table schema:
y: int64
-- schema metadata --
pandas: '{"index_columns": [{"kind": "range", "name": null, "start": 0, "' + 358
Data Schema:
x: int64

## Cleanup

In [ ]:
!rm -rf tmp